In [5]:
!pip install plotly panel

Defaulting to user installation because normal site-packages is not writeable


In [14]:
pip install dash plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
#DATA ANALYSIS PROYECT
# GROUP 2
# Description: Extraction and cleaning of hourly data from the ESIOS API for the following indicators
# - Forecast of wind power production (ID 541)
# - Actual total wind power generation (ID 551)

import requests
import pandas as pd
from datetime import datetime, timedelta

# PERSONAL TOKEN CONFIGURATION
TOKEN = '255c4529289ed8e7cfcfdc5cff2c43d0f101fe5b3adaa20273c01b0deafa80d4'
HEADERS = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'x-api-key': TOKEN,
    'User-Agent': 'esios-api-client'
}

# DATE CONFIGURATION
end = datetime.utcnow()
start = end - timedelta(days=2)  

# GENERAL FUNCTION TO QUERY HOURLY INDICATORS
def get_esios_data(indicator_id, start_date, end_date):
    url = f'https://api.esios.ree.es/indicators/{indicator_id}'
    params = {
        'start_date': start_date.isoformat(),
        'end_date': end_date.isoformat(),
        'time_trunc': 'hour'
    }
    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code == 200:
        data = response.json()
        values = data['indicator']['values']
        df = pd.DataFrame(values)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df[['datetime', 'value']].rename(columns={'value': f'indicator_{indicator_id}'})
        return df
    else:
        print(f"Error {response.status_code}: {response.text}")
        return pd.DataFrame(columns=['datetime', f'indicator_{indicator_id}'])

# DATA DOWNLOAD
df_forecast = get_esios_data(541, start, end)   # Forecast
print("Previsión descargada:")
print(df_forecast.head())

df_real = get_esios_data(551, start, end)       # Real
print("Producción real descargada:")
print(df_real.head())

# MERGE
df = pd.merge(df_forecast, df_real, on='datetime', how='outer').sort_values('datetime')

# DATA CLEANING
# Fill missing values using time-based interpolation (requires indexing by datetime)
df.set_index('datetime', inplace=True)
df.interpolate(method='time', inplace=True)
df.reset_index(inplace=True)

# Outlier removal using the IQR method
for col in ['indicator_541', 'indicator_551']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].where(df[col].between(lower_bound, upper_bound))

# EXPORT TO CSV
df.to_csv("WIND_DATA.csv", index=False)
print("\nDatos limpios exportados a 'WIND_DATA.csv'")

## CÁLCULO DE LA CORRELACIÓN DE LOS DATOS (VERIFICACIÓN ALTERNATIVA)

import numpy as np

# Elimina filas con NaN en las dos columnas relevantes
df_corr = df.dropna(subset=['indicator_541', 'indicator_551'])

# Extrae los valores como arrays de numpy
x = df_corr['indicator_541'].values
y = df_corr['indicator_551'].values

# Calcula la correlación con numpy
correlation = np.corrcoef(x, y)[0, 1]
print(correlation)


from dash import Dash, html, dcc
import plotly.graph_objects as go
import numpy as np

# === DASHBOARD INTERACTIVO ===

# Gráfico de líneas
fig_lines = go.Figure()
fig_lines.add_trace(go.Scatter(x=df['datetime'], y=df['indicator_541'],
                               mode='lines', name='Forecast Wind Power (MW)'))
fig_lines.add_trace(go.Scatter(x=df['datetime'], y=df['indicator_551'],
                               mode='lines', name='Real Wind Power (MW)'))
fig_lines.update_layout(
    title='Wind Power Forecast vs Real',
    xaxis_title='Datetime',
    yaxis_title='Power (MW)',
    template='plotly_dark',
    hovermode='x unified'
)

# Gráfico de dispersión con línea de tendencia
m, b = np.polyfit(x, y, 1)
fig_scatter = go.Figure()
fig_scatter.add_trace(go.Scatter(x=x, y=y, mode='markers',
                                 marker=dict(color='royalblue', size=5, opacity=0.6),
                                 name='Data Points'))
fig_scatter.add_trace(go.Scatter(x=x, y=m * x + b, mode='lines',
                                 name='Trend Line',
                                 line=dict(color='firebrick')))
fig_scatter.update_layout(
    title=f'Correlation Forecast vs Real (r = {correlation:.2f})',
    xaxis_title='Forecast Wind Power (MW)',
    yaxis_title='Real Wind Power (MW)',
    template='plotly_white'
)

# App Dash embebida al final
app = Dash(__name__)
app.title = "Wind Dashboard"

app.layout = html.Div([
    html.H1("Wind Power Dashboard", style={'textAlign': 'center'}),
    dcc.Graph(figure=fig_lines),
    dcc.Graph(figure=fig_scatter),
    html.Div(f"Correlación: r = {correlation:.3f}",
             style={'textAlign': 'center', 'fontSize': '18px', 'marginBottom': '20px'})
])

if __name__ == '__main__':
    app.run(debug=True)


C:\Users\Alumno\AppData\Local\Temp\ipykernel_9580\3694043972.py:21: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



Previsión descargada:
                   datetime  indicator_541
0 2025-05-17 11:00:00+02:00         2078.0
1 2025-05-17 12:00:00+02:00         4294.0
2 2025-05-17 13:00:00+02:00         4391.0
3 2025-05-17 14:00:00+02:00         4545.0
4 2025-05-17 15:00:00+02:00         5052.0
Producción real descargada:
                   datetime  indicator_551
0 2025-05-17 11:00:00+02:00         6349.0
1 2025-05-17 12:00:00+02:00         9516.0
2 2025-05-17 13:00:00+02:00         7198.0
3 2025-05-17 14:00:00+02:00         7293.0
4 2025-05-17 15:00:00+02:00         7534.0

Datos limpios exportados a 'WIND_DATA.csv'
0.9268341063047308
